# <center> <img src="../../labs/img/ITESOLogo.png" alt="ITESO" width="480" height="130"> </center>
# <center> **Departamento de Electrónica, Sistemas e Informática** </center>
---
## <center> **Procesamiento de Datos Masivos** </center>
---
### <center> **Primavera 2025** </center>
---
### <center> **Ejemplos de Spark: Structured Streaming (Kafka consumer application)** </center>

---
**Profesor**: Dr. Pablo Camarillo Ramirez

In [ ]:
import findspark
from equipo.spark_utils import SparkUtils
from pyspark.sql.functions import from_json


findspark.init()

ModuleNotFoundError: No module named 'pyspark'

In [ ]:
SPARK_ID = "5af50e5e22eb"
KAFKA_ID = "fdb4e6c32fcb"

#### Creacion de la conexión con el cluster de spark


In [28]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Equipo-kafka-structured-stream") \
    .master(f"spark://{SPARK_ID}:7077") \
    .config("spark.ui.port","4040") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.13:3.5.4") \
    .getOrCreate()
sc = spark.sparkContext

### Creación del Kafka Stream

In [29]:
kafka_lines = spark \
                .readStream \
                .format("kafka") \
                .option("kafka.bootstrap.servers", f"{KAFKA_ID}:9093") \
                .option("subscribe", "tweet-1, tweet-2, tweet-3, tweet-4") \
                .load()

kafka_lines.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [30]:
headers = [
        ("tweet_id", "string"),
        ("user_id", "integer"),
        ("timestamp", "string"),
        ("text", "string"),
        ("hashtags", "string"),
        ("mentions", "string"),
        ("retweet_count", "integer"),
        ("favorite_count", "integer"),
        ("reply_count", "integer"),
        ("quote_count", "integer"),
        ("views", "integer")
]

tweet_schema = SparkUtils.generate_schema(headers)

### Transform binary data into string

In [31]:
kafka_df = kafka_lines.withColumn("value_str", kafka_lines.value.cast("string"))
kafka_df = kafka_df.select(from_json(kafka_df.value_str, tweet_schema).alias("data")).select("data.*")

### Configuración del "Sink" del stream

In [ ]:

query = kafka_df \
                .writeStream \
                .outputMode("append") \
                .trigger(processingTime='5 seconds') \
                .format("parquet") \
                .option("path", "/home/jovyan/notebooks/data/parquet") \
                .option("truncate", "false") \
                .option("checkpointLocation", "/home/jovyan/notebooks/data/checkpoint") \
                .start()

query.awaitTermination(300)


25/05/13 00:39:47 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
25/05/13 00:39:50 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


False

In [33]:
query.stop()
sc.stop()